In [187]:
import boto3
import pandas as pd
from datetime import datetime
import re
import numpy as np
import time
import difflib
from fuzzywuzzy import process


# Load all_tournaments
all_tournaments_with_stats = pd.read_csv('data/all_tournaments_with_stats2020.csv').dropna(subset=['player0', 'player1']).reset_index()

# Load all_odds
all_tournaments_with_stats.head()

,level_0,index,name,phase,year,duree,surfaceInout,typeTournament,date,winnerRate,...,serviceGamePlayed1,serviceGamesWon1,startCareer1,surface1,totalPointsWon1,totalServicePointsWon1,tournCell1,url1,weight1,year1
0,0,0,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,...,15,93.0,2017,hard,52.0,70.0,25.0,https://www.atptour.com/en/players/alexei-popy...,172.0,2020.0
1,1,1,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,...,114,77.0,2009,hard,49.0,62.0,25.0,https://www.atptour.com/en/players/henri-laaks...,172.0,2020.0
2,2,2,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,...,17,59.0,2016,hard,43.0,56.0,30.0,https://www.atptour.com/en/players/elliot-benc...,194.0,2020.0
3,3,3,montpellier,Round of 32,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,...,[],NaN,2012,hard,NaN,NaN,26.0,https://www.atptour.com/en/players/quentin-hal...,172.0,2020.0
4,5,5,montpellier,Round of 16,2020,6,Indoor Hard,250,2020.02.03 - 2020.02.09,100,...,42,74.0,2009,hard,53.0,61.0,27.0,https://www.atptour.com/en/players/david-goffi...,154.0,2020.0


In [188]:
all_tournaments_with_stats_drop = all_tournaments_with_stats.drop(['index', 'name', 'url', 'date', 'url0','url1','year0',
                                                                  'year1','surface0','surface1','level_0'], axis=1)
all_tournaments_with_stats_drop['plays0']=[play.lstrip().rstrip() for play in all_tournaments_with_stats_drop.plays0]

all_tournaments_with_stats_drop['plays1']=[play.lstrip().rstrip() for play in all_tournaments_with_stats_drop.plays1]

all_tournaments_with_stats_drop.breakPointsConverted0 = all_tournaments_with_stats_drop.breakPointsConverted0.replace('[]',np.NaN)
all_tournaments_with_stats_drop.breakPointsFaced0 = all_tournaments_with_stats_drop.breakPointsFaced0.replace('[]',np.NaN)
all_tournaments_with_stats_drop.breakPointsOpportunities0 = all_tournaments_with_stats_drop.breakPointsOpportunities0.replace('[]',np.NaN)
all_tournaments_with_stats_drop.breakPointsSaved0 = all_tournaments_with_stats_drop.breakPointsSaved0.replace('[]',np.NaN)
all_tournaments_with_stats_drop.doubleFautes0 = all_tournaments_with_stats_drop.doubleFautes0.replace('[]',np.NaN)
all_tournaments_with_stats_drop.firstServe0 = all_tournaments_with_stats_drop.firstServe0.replace('[]',np.NaN)
all_tournaments_with_stats_drop.firstServePointsWon0 = all_tournaments_with_stats_drop.firstServePointsWon0.replace('[]',np.NaN)
all_tournaments_with_stats_drop.firstServeReturnPointsWon0 = all_tournaments_with_stats_drop.firstServeReturnPointsWon0.replace('[]',np.NaN)
all_tournaments_with_stats_drop.breakPointsConverted1 = all_tournaments_with_stats_drop.breakPointsConverted1.replace('[]',np.NaN)
all_tournaments_with_stats_drop.breakPointsFaced1 = all_tournaments_with_stats_drop.breakPointsFaced1.replace('[]',np.NaN)
all_tournaments_with_stats_drop.breakPointsOpportunities1 = all_tournaments_with_stats_drop.breakPointsOpportunities1.replace('[]',np.NaN)
all_tournaments_with_stats_drop.breakPointsSaved1 = all_tournaments_with_stats_drop.breakPointsSaved1.replace('[]',np.NaN)
all_tournaments_with_stats_drop.doubleFautes1 = all_tournaments_with_stats_drop.doubleFautes1.replace('[]',np.NaN)
all_tournaments_with_stats_drop.firstServe1 = all_tournaments_with_stats_drop.firstServe1.replace('[]',np.NaN)
all_tournaments_with_stats_drop.firstServePointsWon1 = all_tournaments_with_stats_drop.firstServePointsWon1.replace('[]',np.NaN)
all_tournaments_with_stats_drop.firstServeReturnPointsWon1 = all_tournaments_with_stats_drop.firstServeReturnPointsWon1.replace('[]',np.NaN)
all_tournaments_with_stats_drop.winnerRate = all_tournaments_with_stats_drop.winnerRate.replace('', 50)
all_tournaments_with_stats_drop.looserRate = all_tournaments_with_stats_drop.looserRate.replace('', 50)
all_tournaments_with_stats_drop.winnerName = all_tournaments_with_stats_drop.winnerName.replace('', np.NaN)
all_tournaments_with_stats_drop.looserName = all_tournaments_with_stats_drop.looserName.replace('', np.NaN)
all_tournaments_with_stats_drop = all_tournaments_with_stats_drop.dropna()

all_tournaments_with_stats_drop.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 327
Data columns (total 72 columns):
phase                          284 non-null object
year                           284 non-null int64
duree                          284 non-null int64
surfaceInout                   284 non-null object
typeTournament                 284 non-null object
winnerRate                     284 non-null int64
looserRate                     284 non-null int64
looserName                     284 non-null object
winnerName                     284 non-null object
winnerRateSurface              284 non-null float64
looserRateSurface              284 non-null float64
label                          284 non-null object
index.1                        284 non-null int64
aces0                          284 non-null object
age0                           284 non-null float64
breakPointsConverted0          284 non-null float64
breakPointsFaced0              284 non-null object
breakPointsOpportunities0    

In [189]:
all_tournaments_with_stats_drop.breakPointsFaced0 = pd.to_numeric(all_tournaments_with_stats_drop.breakPointsFaced0)
all_tournaments_with_stats_drop.breakPointsConverted0 = pd.to_numeric(all_tournaments_with_stats_drop.breakPointsConverted0)
all_tournaments_with_stats_drop.returnGamesWon0 = pd.to_numeric(all_tournaments_with_stats_drop.returnGamesWon0)
all_tournaments_with_stats_drop.serviceGamesWon0 = pd.to_numeric(all_tournaments_with_stats_drop.serviceGamesWon0)

all_tournaments_with_stats_drop.breakPointsFaced1 = pd.to_numeric(all_tournaments_with_stats_drop.breakPointsFaced1)
all_tournaments_with_stats_drop.breakPointsConverted1 = pd.to_numeric(all_tournaments_with_stats_drop.breakPointsConverted1)
all_tournaments_with_stats_drop.returnGamesWon1 = pd.to_numeric(all_tournaments_with_stats_drop.returnGamesWon1)
all_tournaments_with_stats_drop.serviceGamesWon1 = pd.to_numeric(all_tournaments_with_stats_drop.serviceGamesWon1)

all_tournaments_with_stats_drop['mental0'] = all_tournaments_with_stats_drop.breakPointsFaced0 + all_tournaments_with_stats_drop.breakPointsConverted0 + all_tournaments_with_stats_drop.returnGamesWon0 + all_tournaments_with_stats_drop.serviceGamesWon0
all_tournaments_with_stats_drop['mental1'] = all_tournaments_with_stats_drop.breakPointsFaced1 + all_tournaments_with_stats_drop.breakPointsConverted1 + all_tournaments_with_stats_drop.returnGamesWon1 + all_tournaments_with_stats_drop.serviceGamesWon1

### Preprocessing for Neural Network

In [190]:
all_tournaments_with_stats_for_DNN = all_tournaments_with_stats_drop.drop(['winnerName','looserName'], axis=1)
all_tournaments_with_stats_for_DNN.typeTournament = all_tournaments_with_stats_for_DNN.typeTournament.replace('finals',2000)
all_tournaments_with_stats_for_DNN.typeTournament = all_tournaments_with_stats_for_DNN.typeTournament.replace('grandslam',2000)

In [191]:
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Qualification',1)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Robin',1)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Round Robin',1)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Round of 128',2)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Round of 64',3)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Round of 32',4)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Round of 16',5)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Quarter-Finals',6)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Semi-Finals',7)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Finals',8)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Quarterfinals',6)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Semifinals',7)
all_tournaments_with_stats_for_DNN.phase = all_tournaments_with_stats_for_DNN.phase.replace('Final',8)

In [192]:
all_tournaments_with_stats_for_DNN['inOut'] = [surface.split(' ')[0] for surface in all_tournaments_with_stats_for_DNN.surfaceInout]
all_tournaments_with_stats_for_DNN['surface'] = [surface.split(' ')[1] for surface in all_tournaments_with_stats_for_DNN.surfaceInout]

all_tournaments_with_stats_for_DNN = pd.concat([all_tournaments_with_stats_for_DNN, 
                                                pd.get_dummies(all_tournaments_with_stats_for_DNN.inOut),
                                                pd.get_dummies(all_tournaments_with_stats_for_DNN.surface)],axis=1)
all_tournaments_with_stats_for_DNN

,phase,year,duree,surfaceInout,typeTournament,winnerRate,looserRate,winnerRateSurface,looserRateSurface,label,...,tournCell1,weight1,mental0,mental1,inOut,surface,Indoor,Outdoor,Clay,Hard
0,5,2020,6,Indoor Hard,250,100,0,1.000000,0.000000,joueur0,...,25.0,172.0,128.0,150.0,Indoor,Hard,1,0,0,1
1,5,2020,6,Indoor Hard,250,100,0,1.000000,0.000000,joueur0,...,25.0,172.0,155.0,182.0,Indoor,Hard,1,0,0,1
2,5,2020,6,Indoor Hard,250,100,0,1.000000,0.000000,joueur0,...,30.0,194.0,317.0,97.0,Indoor,Hard,1,0,0,1
4,5,2020,6,Indoor Hard,250,100,0,1.000000,0.000000,joueur1,...,27.0,154.0,155.0,177.0,Indoor,Hard,1,0,0,1
5,1,2020,6,Indoor Hard,250,0,0,1.000000,0.000000,joueur0,...,19.0,192.0,97.0,118.0,Indoor,Hard,1,0,0,1
6,5,2020,6,Indoor Hard,250,100,0,1.000000,0.000000,joueur1,...,23.0,174.0,153.0,0.0,Indoor,Hard,1,0,0,1
7,4,2020,6,Indoor Hard,250,0,0,1.000000,0.000000,joueur0,...,18.0,190.0,0.0,167.0,Indoor,Hard,1,0,0,1
8,5,2020,6,Indoor Hard,250,0,0,1.000000,0.000000,joueur0,...,10.0,181.0,0.0,95.0,Indoor,Hard,1,0,0,1
9,5,2020,6,Indoor Hard,250,0,0,1.000000,0.000000,joueur0,...,27.0,176.0,119.0,227.0,Indoor,Hard,1,0,0,1
10,1,2020,6,Indoor Hard,250,0,0,1.000000,0.000000,joueur0,...,15.0,172.0,119.0,40.0,Indoor,Hard,1,0,0,1


In [193]:
all_tournaments_with_stats_for_DNN = all_tournaments_with_stats_for_DNN.drop(['surfaceInout'], axis=1)

In [194]:
all_tournaments_with_stats_for_DNN.plays0 = all_tournaments_with_stats_for_DNN.plays0.replace('Eric Lambert', 'Right-Handed, Two-Handed Backhand')
all_tournaments_with_stats_for_DNN.plays0 = all_tournaments_with_stats_for_DNN.plays0.replace('', 'Right-Handed, Two-Handed Backhand')

all_tournaments_with_stats_for_DNN.plays1 = all_tournaments_with_stats_for_DNN.plays1.replace('Brent Larkham', 'Right-Handed, Two-Handed Backhand')
all_tournaments_with_stats_for_DNN.plays1 = all_tournaments_with_stats_for_DNN.plays1.replace('Eric Lambert', 'Right-Handed, Two-Handed Backhand')
all_tournaments_with_stats_for_DNN.plays1 = all_tournaments_with_stats_for_DNN.plays1.replace('Nelen Brodar', 'Right-Handed, Two-Handed Backhand')
all_tournaments_with_stats_for_DNN.plays1 = all_tournaments_with_stats_for_DNN.plays1.replace('Tony Graham', 'Right-Handed, Two-Handed Backhand')
all_tournaments_with_stats_for_DNN.plays1 = all_tournaments_with_stats_for_DNN.plays1.replace('', 'Right-Handed, Two-Handed Backhand')

all_tournaments_with_stats_for_DNN.points0 = all_tournaments_with_stats_for_DNN.points0.replace(',', '.')
all_tournaments_with_stats_for_DNN.points1 = all_tournaments_with_stats_for_DNN.points1.replace(',', '.')


In [195]:
all_tournaments_with_stats_for_DNN.plays0 = [play.split(',')[0] for play in all_tournaments_with_stats_for_DNN.plays0]
all_tournaments_with_stats_for_DNN.plays1 = [play.split(',')[0] for play in all_tournaments_with_stats_for_DNN.plays1]

dummiesPlays0 = pd.get_dummies(all_tournaments_with_stats_for_DNN.plays0)
dummiesPlays0.columns = ['Left-Handed', 'Right-Handed']

dummiesPlays1 = pd.get_dummies(all_tournaments_with_stats_for_DNN.plays1)
dummiesPlays1.columns = ['Left-Handed.1', 'Right-Handed.1']

all_tournaments_with_stats_for_DNN = pd.concat([all_tournaments_with_stats_for_DNN, 
                                                dummiesPlays0,
                                                dummiesPlays1],axis=1)

In [196]:
all_tournaments_with_stats_for_DNN = all_tournaments_with_stats_for_DNN.drop(['plays0','plays1'], axis=1)

all_tournaments_with_stats_for_DNN.points0 = [point.replace(",",".") for point in all_tournaments_with_stats_for_DNN.points0]
all_tournaments_with_stats_for_DNN.points1 = [point.replace(",",".") for point in all_tournaments_with_stats_for_DNN.points1]



In [197]:
player0_face2face = []
player1_face2face = []


for (label, W, L, Wsurface, Lsurface) in zip(all_tournaments_with_stats_for_DNN.label, all_tournaments_with_stats_for_DNN.winnerRate, all_tournaments_with_stats_for_DNN.looserRate, all_tournaments_with_stats_for_DNN.winnerRateSurface, all_tournaments_with_stats_for_DNN.looserRateSurface):
    if label == 0:
        player0_face2face.append([W, Wsurface])
        player1_face2face.append([L, Lsurface])
    else:
        player0_face2face.append([W, Wsurface])
        player1_face2face.append([L, Lsurface])
        
all_tournaments_with_stats_for_DNN = pd.concat([all_tournaments_with_stats_for_DNN,
                       pd.DataFrame(player0_face2face, columns=['RateFace2Face0','RateFace2FaceSurface0']),
                       pd.DataFrame(player1_face2face, columns=['RateFace2Face1','RateFace2FaceSurface1'])], axis=1)


In [198]:
all_tournaments_with_stats_for_DNN.RateFace2Face0 = all_tournaments_with_stats_for_DNN.RateFace2Face0/100 
all_tournaments_with_stats_for_DNN.RateFace2Face1 = all_tournaments_with_stats_for_DNN.RateFace2Face1/100 
all_tournaments_with_stats_for_DNN.ranking0 = all_tournaments_with_stats_for_DNN.ranking0.fillna(300) 
all_tournaments_with_stats_for_DNN.ranking1 = all_tournaments_with_stats_for_DNN.ranking1.fillna(300) 

In [199]:

def getGapRank(rank0, rank1):
    beta = abs(rank0-rank1)/300
    res = [(1-beta)/2,(1+beta)/2]
    return max(res),min(res)



In [200]:
RateFace2Face0All = []
RateFace2FaceSurface0All = []
RateFace2Face1All = []
RateFace2FaceSurface1All = []

counter = 0

for RateFace2Face0,RateFace2FaceSurface0,RateFace2Face1,RateFace2FaceSurface1, ranking0,ranking1 in zip(all_tournaments_with_stats_for_DNN.RateFace2Face0,
                                                                                     all_tournaments_with_stats_for_DNN.RateFace2FaceSurface0,
                                                                                     all_tournaments_with_stats_for_DNN.RateFace2Face1,
                                                                                     all_tournaments_with_stats_for_DNN.RateFace2FaceSurface1,
                                                                                     all_tournaments_with_stats_for_DNN.ranking0,
                                                                                     all_tournaments_with_stats_for_DNN.RateFace2FaceSurface0):
    if np.isnan(ranking0):
        ranking0 = 300
    
    if np.isnan(ranking1):
        ranking1 = 300
    
    if np.isnan(RateFace2Face0) or np.isnan(RateFace2Face1):
        
        ranking = getGapRank(ranking0, ranking1)
        player0ranking = ranking[0]
        player1ranking = ranking[1]
        
        counter += 1
        
        
        if ranking1<ranking0:
            player0ranking = ranking[1]
            player1ranking = ranking[0]
            
        RateFace2Face0All.append(player0ranking)
        RateFace2Face1All.append(player1ranking)
    
    else:
        RateFace2Face0All.append(RateFace2Face0)
        RateFace2Face1All.append(RateFace2Face1)    
        
    if np.isnan(RateFace2FaceSurface0) or np.isnan(RateFace2FaceSurface1):
        
        ranking = getGapRank(ranking0, ranking1)
        player0ranking = ranking[0]
        player1ranking = ranking[1]
        
        if ranking1<ranking0:
            player0ranking = ranking[1]
            player1ranking = ranking[0]
            
        RateFace2FaceSurface0All.append(player0ranking)
        RateFace2FaceSurface1All.append(player1ranking)
    else:
        RateFace2FaceSurface0All.append(RateFace2FaceSurface0)
        RateFace2FaceSurface1All.append(RateFace2FaceSurface1)

In [201]:
all_tournaments_with_stats_for_DNN = all_tournaments_with_stats_for_DNN.drop(['RateFace2FaceSurface0','RateFace2FaceSurface1',
                                                                                      'RateFace2Face0','RateFace2Face1'], axis=1)

all_tournaments_with_stats_for_DNN = pd.concat([all_tournaments_with_stats_for_DNN.reset_index(),
                                                    pd.DataFrame(RateFace2FaceSurface0All, columns=['RateFace2FaceSurface0']),
                                                    pd.DataFrame(RateFace2FaceSurface1All, columns=['RateFace2FaceSurface1']),
                                                    pd.DataFrame(RateFace2Face0All, columns=['RateFace2Face0']),
                                                    pd.DataFrame(RateFace2Face1All, columns=['RateFace2Face1'])],
                                                    axis=1)
all_tournaments_with_stats_for_DNN

,index,phase,year,duree,typeTournament,winnerRate,looserRate,winnerRateSurface,looserRateSurface,label,...,Clay,Hard,Left-Handed,Right-Handed,Left-Handed.1,Right-Handed.1,RateFace2FaceSurface0,RateFace2FaceSurface1,RateFace2Face0,RateFace2Face1
0,0,5.0,2020.0,6.0,250,100.0,0.0,1.000000,0.000000,joueur0,...,0.0,1.0,1.0,0.0,0.0,1.0,1.000000,0.000000,1.000000,0.000000
1,1,5.0,2020.0,6.0,250,100.0,0.0,1.000000,0.000000,joueur0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,1.000000,0.000000
2,2,5.0,2020.0,6.0,250,100.0,0.0,1.000000,0.000000,joueur0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,1.000000,0.000000
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,0.000000,1.000000,0.000000
4,4,5.0,2020.0,6.0,250,100.0,0.0,1.000000,0.000000,joueur1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,0.000000,0.000000
5,5,1.0,2020.0,6.0,250,0.0,0.0,1.000000,0.000000,joueur0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,1.000000,0.000000
6,6,5.0,2020.0,6.0,250,100.0,0.0,1.000000,0.000000,joueur1,...,0.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,0.000000,0.000000
7,7,4.0,2020.0,6.0,250,0.0,0.0,1.000000,0.000000,joueur0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,0.000000,0.000000
8,8,5.0,2020.0,6.0,250,0.0,0.0,1.000000,0.000000,joueur0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,0.000000,0.000000
9,9,5.0,2020.0,6.0,250,0.0,0.0,1.000000,0.000000,joueur0,...,0.0,1.0,0.0,1.0,0.0,1.0,1.000000,0.000000,0.000000,0.000000


In [204]:
def count(lst): 
  
    return sum(bool(x) for x in lst) 

def getFormPlayer(player, year, df):
    
    counter = 0
    df = df[(df.player0 == player) | (df.player1 == player)]
    df = df[df.year<=year]
    forme = 0
    
    
    if len(df)>3:
        try:
            df = df.iloc[:3]
            rank0 = df[((df.label == 1) & (df.player0 != player))].ranking0
            rank1 = df[((df.label == 0) & (df.player1 != player))].ranking1
            df = df[((df.label == 0) & (df.player0 == player)) | ((df.label == 1) & (df.player1 == player))]
            forme = len(df)/3
            rank = (rank0.mean()+rank1.mean())/(len(rank0)+len(rank1))
            return forme, rank
        except:
            return 0.5,150
        
    else:
        try:
            rank0 = df[((df.label == 1) & (df.player0 != player))].ranking0
            rank1 = df[((df.label == 0) & (df.player1 != player))].ranking1
            initLenDf = len(df)
            df = df[((df.label == 0) & (df.player0 == player)) | ((df.label == 1) & (df.player1 == player))]
            forme = len(df)/initLenDf
            rank = (rank0.mean()+rank1.mean())/(len(rank0)+len(rank1))
            return forme, rank
        except:
            return 0,0


In [178]:
Allforme0 = []
Allforme1 = []
AllrankingMean0 = []
AllrankingMean1 = []


for player0, player1, year in zip(all_tournaments_with_stats_for_DNN.player0,
                                  all_tournaments_with_stats_for_DNN.player1,
                                  all_tournaments_with_stats_for_DNN.year):
    
    
    
    
    
    forme0, rank0 = getFormPlayer(player0, year, all_tournaments_with_stats_for_DNN)
    forme1, rank1 = getFormPlayer(player1, year, all_tournaments_with_stats_for_DNN)
        
    Allforme0.append(forme0)
    Allforme1.append(forme1)
    AllrankingMean0.append(rank0)
    AllrankingMean1.append(rank1)
    
all_tournaments_with_stats_for_DNN = pd.concat([all_tournaments_with_stats_for_DNN.reset_index(drop=True),
                                                    pd.DataFrame(Allforme0, columns=['Forme0']),
                                                    pd.DataFrame(Allforme1, columns=['Forme1']),
                                                    pd.DataFrame(AllrankingMean0, columns=['WinMean0']),
                                                    pd.DataFrame(AllrankingMean1, columns=['WinMean1'])],
                                                    axis=1)

if 'Carpet' not in all_tournaments_with_stats_for_DNN.columns:
    all_tournaments_with_stats_for_DNN = pd.concat([all_tournaments_with_stats_for_DNN.reset_index(drop=True),
                                                    pd.DataFrame(np.zeros(len(all_tournaments_with_stats_for_DNN)), columns=['Carpet'])],
                                                    axis=1)
    
if 'Grass' not in all_tournaments_with_stats_for_DNN.columns:
    all_tournaments_with_stats_for_DNN = pd.concat([all_tournaments_with_stats_for_DNN.reset_index(drop=True),
                                                    pd.DataFrame(np.zeros(len(all_tournaments_with_stats_for_DNN)), columns=['Grass'])],
                                                    axis=1)

In [179]:
all_historic_for_DNN = pd.read_csv('data/all_tournaments_with_stats_for_DNN.csv')
all_tournaments_with_stats_for_DNN = all_tournaments_with_stats_for_DNN[all_historic_for_DNN.columns]
all_tournaments_with_stats_for_DNN.ranking0 = all_tournaments_with_stats_for_DNN.ranking0.fillna(300) 
all_tournaments_with_stats_for_DNN.ranking1 = all_tournaments_with_stats_for_DNN.ranking1.fillna(300)
all_tournaments_with_stats_for_DNN.WinMean0 = all_tournaments_with_stats_for_DNN.WinMean0.fillna(200) 
all_tournaments_with_stats_for_DNN.WinMean1 = all_tournaments_with_stats_for_DNN.WinMean1.fillna(200) 
all_tournaments_with_stats_for_DNN.to_csv('data/all_tournaments_with_stats_for_DNN2020.csv', index=False)

/Users/matrousseau/.local/share/virtualenvs/Pronoo-RxTA2Kor/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (37,65) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Index(['index', 'phase', 'year', 'duree', 'typeTournament', 'winnerRate',
       'looserRate', 'winnerRateSurface', 'looserRateSurface', 'label',
       'aces0', 'age0', 'breakPointsConverted0', 'breakPointsFaced0',
       'breakPointsOpportunities0', 'breakPointsSaved0', 'doubleFautes0',
       'firstServe0', 'firstServePointsWon0', 'firstServeReturnPointsWon0',
       'height0', 'nextBest0', 'player0', 'points0', 'pointsDropping0',
       'ranking0', 'returnGamesPlayed0', 'returnGamesWon0', 'returnPointsWon0',
       'secondServePointsWon0', 'secondServeReturnPointsWon0',
       'serviceGamePlayed0', 'serviceGamesWon0', 'startCareer0',
       'totalPointsWon0', 'totalServicePointsWon0', 'tournCell0', 'weight0',
       'aces1', 'age1', 'breakPointsConverted1', 'breakPointsFaced1',
       'breakPointsOpportunities1', 'breakPointsSaved1', 'doubleFautes1',
       'firstServe1', 'firstServePointsWon1', 'firstServeReturnPointsWon1',
       'height1', 'nextBest1', 'player1', 'points1', 'poi